### Model v1

This version of modeling is with no features. This notebook tries several different models with same dataset.

In [1]:
import pandas as pd

games_df = pd.read_csv("../data/processed/data.csv")
games_df

,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi
0,Lin Lixiang,Lin Xiuping,b,w,2020-01-14,7p,6p,6.5
1,Jian Jingting,Hei Jiajia,b,w,2018-03-19,3p,7p,6.5
2,Toyoda Hirohito,Aoki Hirotaka,b,w,2021-09-27,1p,2p,6.5
3,Shimamura Toshihiro,Hashimoto Utaro,w,b,1963-01-16,9p,9p,4.5
4,Oh Yujin,Zhao Yifei,w,b,2018-08-14,6p,3p,7.5
...,...,...,...,...,...,...,...,...
101404,Gao Xing,Gu Wanshan,w,b,2021-09-27,4p,2p,7.5
101405,Kang Dongyun,Lin Junyan,b,w,2012-12-12,9p,3p,7.5
101406,Shin Jinseo,Park Junghwan,b,w,2020-10-22,9p,9p,6.5
101407,Wang Yuhui,Chen Linxin,b,w,1999-11-11,6p,9p,5.5


In [2]:
labeled_df = games_df.drop(['win_name', 'lose_name', 'date'], axis=1)
labeled_df = labeled_df.rename(columns={'win_color': 'player_color', 'lose_color': 'opponent_color', 'win_rank': 'player_rank', 'lose_rank': 'opponent_rank'})
labeled_df['target'] = 1

labeled_df

,player_color,opponent_color,player_rank,opponent_rank,komi,target
0,b,w,7p,6p,6.5,1
1,b,w,3p,7p,6.5,1
2,b,w,1p,2p,6.5,1
3,w,b,9p,9p,4.5,1
4,w,b,6p,3p,7.5,1
...,...,...,...,...,...,...
101404,w,b,4p,2p,7.5,1
101405,b,w,9p,3p,7.5,1
101406,b,w,9p,9p,6.5,1
101407,b,w,6p,9p,5.5,1


In [3]:
reversed_df = labeled_df.copy(deep=True)
reversed_df = reversed_df.rename(columns={'player_color':'opponent_color', 'opponent_color':'player_color', 'player_rank':'opponent_rank', 'opponent_rank':'player_rank'})
reversed_df['target'] = 0

reversed_df

,opponent_color,player_color,opponent_rank,player_rank,komi,target
0,b,w,7p,6p,6.5,0
1,b,w,3p,7p,6.5,0
2,b,w,1p,2p,6.5,0
3,w,b,9p,9p,4.5,0
4,w,b,6p,3p,7.5,0
...,...,...,...,...,...,...
101404,w,b,4p,2p,7.5,0
101405,b,w,9p,3p,7.5,0
101406,b,w,9p,9p,6.5,0
101407,b,w,6p,9p,5.5,0


In [4]:
processed_df = pd.concat([labeled_df,reversed_df], ignore_index=True)
processed_df = processed_df[(~processed_df['player_rank'].isin(['Ama.', 'Insei', 'NR'])) & (~processed_df['opponent_rank'].isin(['Ama.', 'Insei', 'NR']))]
processed_df['player_rank'] = processed_df['player_rank'].apply(lambda x: int(x[0]))
processed_df['opponent_rank'] = processed_df['opponent_rank'].apply(lambda x: int(x[0]))
processed_df['player_color'] = processed_df['player_color'].apply(lambda x: 1 if x == 'b' else 0)
processed_df['opponent_color'] = processed_df['opponent_color'].apply(lambda x: 1 if x == 'w' else 0)

processed_df


,player_color,opponent_color,player_rank,opponent_rank,komi,target
0,1,1,7,6,6.5,1
1,1,1,3,7,6.5,1
2,1,1,1,2,6.5,1
3,0,0,9,9,4.5,1
4,0,0,6,3,7.5,1
...,...,...,...,...,...,...
202813,1,1,2,4,7.5,0
202814,0,0,3,9,7.5,0
202815,0,0,9,9,6.5,0
202816,0,0,9,6,5.5,0


In [5]:
from sklearn.model_selection import train_test_split

X, y = processed_df[['player_color', 'opponent_color', 'player_rank', 'opponent_rank', 'komi']], processed_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Random Forest Classification

In [6]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.5577106328912889

We only get 55% accuracy. This makes sense though. We only have player and opponent's rank plus the komi, to make predictions based off of. I won't really try further with different models.